In [8]:
import xarray as xr
import pandas as pd
import numpy as np

Data AMH Observasi dari data CHIRPS

In [9]:
AMH = xr.open_dataset('AMHfull.nc')
rainy_season = AMH['AMH']
RS = rainy_season[1:31,:,:] #mengambil data dari tahun 1982-2011

Mempersiapkan data setiap model dan lead time nya

Data Model NMME CFSv2 dengan lead time bulan Januari-Mei

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

# Daftar file dan bulan yang akan diproses
file_dict = {
    'Januari': 'dataCFS1.nc',
    'Februari': 'dataCFS2.nc',
    'Maret': 'dataCFS3.nc',
    'April': 'dataCFS4.nc',
    'Mei': 'dataCFS5.nc'
}

# Dimensi yang akan diubah namanya
dim_names_new = {'S': 'time', 'Y': 'latitude', 'X': 'longitude'}

# Rentang grid untuk interpolasi
regrid_lat = np.arange(-4.625, 5.130, 0.05)
regrid_lon = np.arange(108.375, 119.130, 0.05)

# Looping untuk setiap bulan dan file
for month, filename in file_dict.items():
    # Membaca dataset untuk bulan tertentu
    ds1 = xr.open_dataset(filename, decode_times=False)
    
    # Menetapkan dimensi waktu
    time_dim = 'S' if 'S' in ds1.dims else 'time'
    ds1[time_dim] = pd.date_range(start=f'1982-{list(file_dict.keys()).index(month)+1:02d}-01', periods=ds1.dims[time_dim], freq='M')

    # Memilih data presipitasi untuk bulan yang sesuai
    rainfall = ds1.sel(S=ds1['S.month'] == (list(file_dict.keys()).index(month) + 1))['prec']
    
    # Hitung presipitasi untuk Agustus, September, Oktober
    Agustus = rainfall[:, 0, :, :] * 31
    September = rainfall[:, 1, :, :] * 30
    Oktober = rainfall[:, 2, :, :] * 31

    # Mengubah data bulan menjadi dataset dan menghapus variabel 'L' jika ada
    Agustus = Agustus.to_dataset(name='prec').drop_vars('L', errors='ignore')
    September = September.to_dataset(name='prec').drop_vars('L', errors='ignore')
    Oktober = Oktober.to_dataset(name='prec').drop_vars('L', errors='ignore')

    # Mengonversi kembali ke variabel presipitasi untuk setiap bulan
    Agustus = Agustus['prec']
    September = September['prec']
    Oktober = Oktober['prec']

    # Membuat dataset gabungan untuk Agustus hingga Oktober
    combined_ASO = xr.Dataset({
        'Agustus': Agustus,
        'September': September,
        'Oktober': Oktober
    })

    # Mengatur dimensi dengan nama baru
    combined_ASO = combined_ASO.rename_dims(dim_names_new)
    combined_ASO = combined_ASO.rename_vars(dim_names_new)

    # Melakukan interpolasi
    regridded = combined_ASO.interp(latitude=regrid_lat, longitude=regrid_lon, method="linear")
    regridded = regridded.groupby('time.year').mean()

    # Membuat dataset akhir yang menggabungkan data Agustus hingga Oktober dan variabel AMH
    Data_AMH = xr.Dataset({
        'Agustus': regridded['Agustus'],
        'September': regridded['September'],
        'Oktober': regridded['Oktober'],
        'AMH': RS  # Pastikan variabel 'RS' telah didefinisikan sebelumnya
    })

    # Menyimpan dataset gabungan ke dalam file NetCDF
    Data_AMH.to_netcdf(f"New_CFS{month}.nc")
    print(f"Data for {month} saved as New_CFS{month}.nc")

Data Model NMME CanSIPS-IC3 dengan lead time bulan Januari-Mei

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

# Daftar file dan bulan yang akan diproses
file_dict = {
    'Januari': 'dataIC31.nc',
    'Februari': 'dataIC32.nc',
    'Maret': 'dataIC33.nc',
    'April': 'dataIC34.nc',
    'Mei': 'dataIC35.nc'
}

# Dimensi yang akan diubah namanya
dim_names_new = {'S': 'time', 'Y': 'latitude', 'X': 'longitude'}

# Rentang grid untuk interpolasi
regrid_lat = np.arange(-4.625, 5.130, 0.05)
regrid_lon = np.arange(108.375, 119.130, 0.05)

# Looping untuk setiap bulan dan file
for month, filename in file_dict.items():
    # Membaca dataset untuk bulan tertentu
    ds1 = xr.open_dataset(filename, decode_times=False)
    
    # Menetapkan dimensi waktu
    time_dim = 'S' if 'S' in ds1.dims else 'time'
    ds1[time_dim] = pd.date_range(start=f'1982-{list(file_dict.keys()).index(month)+1:02d}-01', periods=ds1.dims[time_dim], freq='M')

    # Memilih data presipitasi untuk bulan yang sesuai
    rainfall = ds1.sel(S=ds1['S.month'] == (list(file_dict.keys()).index(month) + 1))['prec']
    
    # Hitung presipitasi untuk Agustus, September, Oktober
    Agustus = rainfall[:, 0, :, :] * 31
    September = rainfall[:, 1, :, :] * 30
    Oktober = rainfall[:, 2, :, :] * 31

    # Mengubah data bulan menjadi dataset dan menghapus variabel 'L' jika ada
    Agustus = Agustus.to_dataset(name='prec').drop_vars('L', errors='ignore')
    September = September.to_dataset(name='prec').drop_vars('L', errors='ignore')
    Oktober = Oktober.to_dataset(name='prec').drop_vars('L', errors='ignore')

    # Mengonversi kembali ke variabel presipitasi untuk setiap bulan
    Agustus = Agustus['prec']
    September = September['prec']
    Oktober = Oktober['prec']

    # Membuat dataset gabungan untuk Agustus hingga Oktober
    combined_ASO = xr.Dataset({
        'Agustus': Agustus,
        'September': September,
        'Oktober': Oktober
    })

    # Mengatur dimensi dengan nama baru
    combined_ASO = combined_ASO.rename_dims(dim_names_new)
    combined_ASO = combined_ASO.rename_vars(dim_names_new)

    # Melakukan interpolasi
    regridded = combined_ASO.interp(latitude=regrid_lat, longitude=regrid_lon, method="linear")
    regridded = regridded.groupby('time.year').mean()

    # Membuat dataset akhir yang menggabungkan data Agustus hingga Oktober dan variabel AMH
    Data_AMH = xr.Dataset({
        'Agustus': regridded['Agustus'],
        'September': regridded['September'],
        'Oktober': regridded['Oktober'],
        'AMH': RS  # Pastikan variabel 'RS' telah didefinisikan sebelumnya
    })

    # Menyimpan dataset gabungan ke dalam file NetCDF
    Data_AMH.to_netcdf(f"New_IC3{month}.nc")
    print(f"Data for {month} saved as New_IC3{month}.nc")

Data Model NMME Cansipv2 dengan lead time bulan Januari-Mei

In [ ]:
import xarray as xr
import pandas as pd
import numpy as np

# Daftar file dan bulan yang akan diproses
file_dict = {
    'Januari': 'dataCansip1.nc',
    'Februari': 'dataCansip2.nc',
    'Maret': 'dataCansip3.nc',
    'April': 'dataCansip4.nc',
    'Mei': 'dataCansip5.nc'
}

# Dimensi yang akan diubah namanya
dim_names_new = {'S': 'time', 'Y': 'latitude', 'X': 'longitude'}

# Rentang grid untuk interpolasi
regrid_lat = np.arange(-4.625, 5.130, 0.05)
regrid_lon = np.arange(108.375, 119.130, 0.05)

# Looping untuk setiap bulan dan file
for month, filename in file_dict.items():
    # Membaca dataset untuk bulan tertentu
    ds1 = xr.open_dataset(filename, decode_times=False)
    
    # Menetapkan dimensi waktu
    time_dim = 'S' if 'S' in ds1.dims else 'time'
    ds1[time_dim] = pd.date_range(start=f'1982-{list(file_dict.keys()).index(month)+1:02d}-01', periods=ds1.dims[time_dim], freq='M')

    # Memilih data presipitasi untuk bulan yang sesuai
    rainfall = ds1.sel(S=ds1['S.month'] == (list(file_dict.keys()).index(month) + 1))['prec']
    
    # Hitung presipitasi untuk Agustus, September, Oktober
    Agustus = rainfall[:, 0, :, :] * 31
    September = rainfall[:, 1, :, :] * 30
    Oktober = rainfall[:, 2, :, :] * 31

    # Mengubah data bulan menjadi dataset dan menghapus variabel 'L' jika ada
    Agustus = Agustus.to_dataset(name='prec').drop_vars('L', errors='ignore')
    September = September.to_dataset(name='prec').drop_vars('L', errors='ignore')
    Oktober = Oktober.to_dataset(name='prec').drop_vars('L', errors='ignore')

    # Mengonversi kembali ke variabel presipitasi untuk setiap bulan
    Agustus = Agustus['prec']
    September = September['prec']
    Oktober = Oktober['prec']

    # Membuat dataset gabungan untuk Agustus hingga Oktober
    combined_ASO = xr.Dataset({
        'Agustus': Agustus,
        'September': September,
        'Oktober': Oktober
    })

    # Mengatur dimensi dengan nama baru
    combined_ASO = combined_ASO.rename_dims(dim_names_new)
    combined_ASO = combined_ASO.rename_vars(dim_names_new)

    # Melakukan interpolasi
    regridded = combined_ASO.interp(latitude=regrid_lat, longitude=regrid_lon, method="linear")
    regridded = regridded.groupby('time.year').mean()

    # Membuat dataset akhir yang menggabungkan data Agustus hingga Oktober dan variabel AMH
    Data_AMH = xr.Dataset({
        'Agustus': regridded['Agustus'],
        'September': regridded['September'],
        'Oktober': regridded['Oktober'],
        'AMH': RS  # Pastikan variabel 'RS' telah didefinisikan sebelumnya
    })

    # Menyimpan dataset gabungan ke dalam file NetCDF
    Data_AMH.to_netcdf(f"New_Cansipv2{month}.nc")
    print(f"Data for {month} saved as New_Cansipv2{month}.nc")